In [195]:
# -*- coding: utf-8 -*-
import argparse
import copy
import json
import os
import pickle
import random
import sys
import re
import numpy as np
import openai

sys.path.append('C:\\Users\\Pluto\\Desktop\\TaDe')
from utils import *

os.environ["http_proxy"] = "http://localhost:7890"
os.environ["https_proxy"] = "http://localhost:7890"

GPT_MODEL = "gpt-4-turbo"  # [gpt-4-turbo-preview]

setOpenAi(keyid = 3)

with open("puzzles.json", "r") as f:
    puzzles = json.load(f)

In [196]:
import random
# 随机选择一个puzzle进行解决
i = random.randint(0, len(puzzles))
question = puzzles[i]['sat']
print(puzzles[i])

{'name': 'TripleZeroSum:2', 'sat': 'def sat(inds: List[int], nums=[-64, -74, -18, -57, 89, -14, -25, 11, -60, -78]):\n    return len(inds) == 3 and sum(nums[i] for i in inds) == 0', 'ans_type': 'List[int]', 'sol_header': 'def sol(nums=[-64, -74, -18, -57, 89, -14, -25, 11, -60, -78]):', 'sol_docstring': '    """\n    Find the indices of three numbers that sum to 0 in a list.\n\n    --- Example input ---\n    [1, 2, 4, -3, 5]\n\n    --- Example output ---\n    [0, 1, 3]\n    """', 'sol_bodies': ['    # \\tilde{O}(n^2) algorithm\n    inv = {n: i for i, n in enumerate(nums)}  # note that later duplicates will override earlier entries\n    for i, n in enumerate(nums):\n        if inv[n] == i:\n            del inv[n]\n        if any((-m - n) in inv for m in nums[:i]):  # found solution!\n            j, m = next((j, m) for j, m in enumerate(nums) if (-m - n) in inv)\n            k = inv[-m - n]\n            return sorted([i, j, k])'], 'module': 'human_eval.py', 'notes': 'Inspired by [HumanEv

In [197]:
def decompose_sql(question):    
    
    question_example = f"""
You will be provided with a Programming Puzzle. Your task is to find an input that will make the program return True.
Here is the puzzle: 
def sat(start: int, k=1, lower=93, seq=[-61, -46, 89, 93, -13, 14, -95, -74, -92, -38, -93, 64, -78, 3, 92, -10, -4, 43, 72, 12, 3, -3, -15, -96, 72, -71, -30, 53, 17, -87, 49, 17, -69, 78, 6, -77, -99, 91, 13, 9, 81, -55, 75, 48, -65, 18, -83, 10, -12, 88, 60, -72, -7, -49, -56, -76, 82, 18, 77, 52, -92, -88, 39, 13, -16, 82, 4, 44, -19, 54, 6, 55, 77, -38, -30, -55, -16]):
    return 0 <= start <= len(seq) - k and sum(seq[start:start + k]) >= lower
    
You have to decompose the puzzle into multiple steps. Carefully consider the granularity of each subtask to ensure that each one is executable.

Answer format: Please follow the format below strictly when answering. No explanation is required.
STEP1 [ step task 1 ]
STEP2 [ step task 2 ]
...
"""
    answer_example = f"""
STEP1 [ Understand the puzzle's primary objective: Find an integer 'start' such that when added to the subsequent 'k' elements in 'seq', the sum is greater than or equal to 'lower'. ]
STEP2 [ Note the inputs: 'k', 'lower', and 'seq' are given and fixed. ]
STEP3 [ Recognize the constraints: The variable 'start' must be within the range 0 to len(seq) - k. ]
STEP4 [ Comprehend that sum(seq[start:start + k]) needs to be greater than or equal to 'lower'. ]
STEP5 [ Deduce a strategy: Loop through the sequence from 0 to len(seq) - k to find a valid 'start'. ]
STEP6 [ For each 'start' value in the loop, calculate sum(seq[start:start + k]). ]
STEP7 [ Compare the calculated sum with 'lower'. ]
STEP8 [ If the sum is greater than or equal to 'lower', return that 'start' value. ]
"""

    Example = [
        {"role": "user", "content": question_example},
        {"role": "assistant", "content": answer_example}
    ]


    prompt_for_decompose = f"""
You will be provided with a Programming Puzzle. Your task is to find an input that will make the program return True.
Here is the puzzle: {question}
You have to decompose the puzzle into multiple steps. Carefully consider the granularity of each subtask to ensure that each one is executable.

Answer format: Please follow the format below strictly when answering. No explanation is required.
STEP1 [ step task 1 ]
STEP2 [ step task 2 ]
...
"""
    Q = {
        "role": "user",
        "content": prompt_for_decompose
    }
    Query = Example+[Q]
    result = askChatGPT(Query, model=GPT_MODEL, temperature=1)
    return result


def convert_steps_to_format(raw_steps):
    lines = raw_steps.strip().split('\n')
    steps_dict = {}
    steps = []
    for line in lines:
        if line.strip():  # 只处理非空行
            step_number = int(line.split(' ')[0][4:])  # 提取数值部分并转换为整数
            step_id = line.split(' ')[0]
            step_content = line[line.index('[') + 1 : line.rindex(']')]
            steps_dict[step_number] = step_content
            steps.append({"stepId": step_number, "step": step_content})
            
            
    # return steps_dict
    return steps, steps_dict

In [198]:
result = decompose_sql(question)
print(result)

STEP1 [ Understand the objective: Find three indices in 'nums' whose values sum up to 0. ]
STEP2 [ Note the inputs: 'nums' is a given and fixed list. ]
STEP3 [ Recognize constraints: 'inds' must have exactly 3 elements. ]
STEP4 [ Comprehend that the sum of nums at those indices must equal 0. ]
STEP5 [ Consider all possible combinations of 3 indices in 'nums'. ]
STEP6 [ For each combination of indices, calculate the sum of values at those indices. ]
STEP7 [ Check if the calculated sum equals 0. ]
STEP8 [ If a valid set of indices is found, return them. ]


In [199]:
steps, steps_dict = convert_steps_to_format(result)
print(steps)

[{'stepId': 1, 'step': " Understand the objective: Find three indices in 'nums' whose values sum up to 0. "}, {'stepId': 2, 'step': " Note the inputs: 'nums' is a given and fixed list. "}, {'stepId': 3, 'step': " Recognize constraints: 'inds' must have exactly 3 elements. "}, {'stepId': 4, 'step': ' Comprehend that the sum of nums at those indices must equal 0. '}, {'stepId': 5, 'step': " Consider all possible combinations of 3 indices in 'nums'. "}, {'stepId': 6, 'step': ' For each combination of indices, calculate the sum of values at those indices. '}, {'stepId': 7, 'step': ' Check if the calculated sum equals 0. '}, {'stepId': 8, 'step': ' If a valid set of indices is found, return them. '}]


建图→化简→可视化

In [200]:
import ipydagred3
import networkx as nx

def construct_dependencies_without_traversal(question, steps):
    prompt = f"Now Given the following subtasks for question: {question}, determine the dependencies between them:\n"
    for step in steps:
        prompt += f"Step {step['stepId']}: {step['step']}\n"
    prompt += """\nPlease list the dependencies in the format 'Subtask A [xxx] -> Subtask B [xxx]' indicating that Subtask A must be completed before Subtask B can start.
Please identify any potential conditional dependencies from a logical perspective.

Answer format: (Please strictly follow the format. Each dependency should be separated by a new line. No explanation is required.)
Step id_i [  subtask i ] -> Step id_j [ subtask j ]
Step id_m [  subtask m ] -> Step id_n [ subtask n ]
...
"""
    prompt_user = {
        "role": "user",
        "content": f"""{prompt}"""
    }
    Query = [prompt_user]
    output = askChatGPT(Query, model=GPT_MODEL, temperature=1)
    return output

def create_dag_from_string(dependencies_string):
    G = nx.DiGraph()
    
    # Split the string by newlines to get individual dependencies
    dependencies = dependencies_string.strip().split('\n')
    
    for dependency in dependencies:
        # Split each dependency by the arrow
        steps = dependency.split(' -> ')
        if len(steps) == 2:
            step_0 = steps[0].strip()
            step_1 = steps[1].strip()
            # Add edges based on dependencies
            G.add_edge(step_0, step_1)
    
    # Print the edges
    # print(list(G.edges()))
    # Perform transitive reduction to simplify the graph
    transitive_reduction = nx.transitive_reduction(G)
    return transitive_reduction

def create_graph(reduced_dependencies, problem):
    graph = ipydagred3.Graph()

    #for state in steps_list:
    #    graph.setNode(state, tooltip='tooltip1 of ' + state)
        
    for dependency in reduced_dependencies:
        graph.setEdge(*dependency)

    graph.setNode(problem, tooltip = "Problem Statement")
        
    widge_try = ipydagred3.DagreD3Widget(graph=graph)
    
    return widge_try

In [201]:
relations_test = construct_dependencies_without_traversal(question, steps)  # query LLM回答所有的依赖
relations_test

"Step 1 [ Understand the objective: Find three indices in 'nums' whose values sum up to 0. ] -> Step 2 [ Note the inputs: 'nums' is a given and fixed list. ]\nStep 1 [ Understand the objective: Find three indices in 'nums' whose values sum up to 0. ] -> Step 3 [ Recognize constraints: 'inds' must have exactly 3 elements. ]\nStep 1 [ Understand the objective: Find three indices in 'nums' whose values sum up to 0. ] -> Step 4 [ Comprehend that the sum of nums at those indices must equal 0. ]\nStep 2 [ Note the inputs: 'nums' is a given and fixed list. ] -> Step 5 [ Consider all possible combinations of 3 indices in 'nums'. ]\nStep 3 [ Recognize constraints: 'inds' must have exactly 3 elements. ] -> Step 6 [ For each combination of indices, calculate the sum of values at those indices. ]\nStep 4 [ Comprehend that the sum of nums at those indices must equal 0. ] -> Step 7 [ Check if the calculated sum equals 0. ]\nStep 5 [ Consider all possible combinations of 3 indices in 'nums'. ] -> Ste

In [202]:
G1 = create_dag_from_string(relations_test)  # LLM 进行建图与化简
reduced_dependencies = list(G1.edges())
print(reduced_dependencies)
create_graph(reduced_dependencies, question)

[("Step 1 [ Understand the objective: Find three indices in 'nums' whose values sum up to 0. ]", "Step 2 [ Note the inputs: 'nums' is a given and fixed list. ]"), ("Step 1 [ Understand the objective: Find three indices in 'nums' whose values sum up to 0. ]", 'Step 4 [ Comprehend that the sum of nums at those indices must equal 0. ]'), ("Step 1 [ Understand the objective: Find three indices in 'nums' whose values sum up to 0. ]", "Step 3 [ Recognize constraints: 'inds' must have exactly 3 elements. ]"), ("Step 2 [ Note the inputs: 'nums' is a given and fixed list. ]", "Step 5 [ Consider all possible combinations of 3 indices in 'nums'. ]"), ("Step 3 [ Recognize constraints: 'inds' must have exactly 3 elements. ]", 'Step 6 [ For each combination of indices, calculate the sum of values at those indices. ]'), ('Step 4 [ Comprehend that the sum of nums at those indices must equal 0. ]', 'Step 7 [ Check if the calculated sum equals 0. ]'), ("Step 5 [ Consider all possible combinations of 3 i

DagreD3Widget()

-------------------------

In [112]:
print(reduced_dependencies)
# 进行一些化简
edges = []
for item in reduced_dependencies:
    edges.append((item[0][:item[0].find('[')].strip(), item[1][:item[1].find('[')].strip()))
print(edges)

int_edges = [(int(e[0].split()[1]), int(e[1].split()[1])) for e in edges]
print(int_edges)


[('Step 1 [ Understand the objective ]', 'Step 3 [ Comprehend the condition to satisfy ]'), ('Step 3 [ Comprehend the condition to satisfy ]', 'Step 6 [ Adjust the computed square root if necessary ]'), ("Step 2 [ Recognize input 'a' is given and fixed ]", 'Step 5 [ Devise a strategy ]'), ('Step 5 [ Devise a strategy ]', 'Step 6 [ Adjust the computed square root if necessary ]'), ('Step 4 [ Recall mathematical knowledge ]', 'Step 5 [ Devise a strategy ]')]
[('Step 1', 'Step 3'), ('Step 3', 'Step 6'), ('Step 2', 'Step 5'), ('Step 5', 'Step 6'), ('Step 4', 'Step 5')]
[(1, 3), (3, 6), (2, 5), (5, 6), (4, 5)]


层级深度计算,似乎是对的

In [113]:
# 计算节点的深度

from collections import deque, defaultdict

def calculate_node_depths(edges):
    # 初始化节点的入度和邻接表
    in_degree = defaultdict(int)
    adjacency_list = defaultdict(list)
    
    nodes = set()
    
    # 填充邻接表和计算入度
    for u, v in edges:
        adjacency_list[u].append(v)
        in_degree[v] += 1
        nodes.add(u)
        nodes.add(v)
    
    # 找到所有入度为0的节点
    queue = deque([node for node in nodes if in_degree[node] == 0])
    depth = {node: 0 for node in queue}
    
    # 广度优先搜索
    while queue:
        node = queue.popleft()
        current_depth = depth[node]
        
        for neighbor in adjacency_list[node]:
            in_degree[neighbor] -= 1
            if in_degree[neighbor] == 0:
                queue.append(neighbor)
                depth[neighbor] = current_depth + 1
    
    return depth

# 示例用法
# edges = [("node1", "node2"), ("node1", "node3"), ("node2", "node4"), ("node3", "node4"), ("node4", "node5")]
node_depths = calculate_node_depths(edges)
print(node_depths)



{'Step 4': 0, 'Step 1': 0, 'Step 2': 0, 'Step 3': 1, 'Step 5': 1, 'Step 6': 2}


开始基于图结构设计推理路径

In [114]:
# 首先推理所有深度为0的点
import sys
sys.path.append('C:\\Users\\Pluto\\Desktop\\TaDe')
from utils import reverseDict, search_Predecessors

# node_depths
depths = reverseDict(node_depths)  # {0: ['Step 1'], 1: ['Step 3', 'Step 2'], 2: ['Step 5', 'Step 4'], 3: ['Step 6'], 4: ['Step 7'], 5: ['Step 8'], 6: ['Step 9']}


In [115]:
print(question)

def sat(x: float, a=7987622700):
    return abs(x ** 2 - a) < 10 ** -3


In [116]:
print(steps_dict)

{1: " Understand the objective: Find a float 'x' such that the difference between 'x' squared and 'a' is less than 0.001. ", 2: " Recognize input 'a' is given and fixed. ", 3: ' Comprehend the condition to satisfy: abs(x ** 2 - a) < 10 ** -3. ', 4: " Recall mathematical knowledge: 'x' is essentially the square root of 'a', within a small tolerance. ", 5: " Devise a strategy: Compute the square root of 'a' as a starting point. ", 6: ' Adjust the computed square root if necessary to ensure the condition abs(x ** 2 - a) < 10 ** -3 is met. '}


In [117]:
print(edges)
print(int_edges)

[('Step 1', 'Step 3'), ('Step 3', 'Step 6'), ('Step 2', 'Step 5'), ('Step 5', 'Step 6'), ('Step 4', 'Step 5')]
[(1, 3), (3, 6), (2, 5), (5, 6), (4, 5)]


In [118]:
import sys
sys.path.append('C:\\Users\\Pluto\\Desktop\\TaDe')
from utils import search_Predecessors


heights = list(depths.keys())
  # print(heights)
MAXHeight = max(heights)
# print(MAXHeight)
answerDict = {}  # 只有已经做过回答的subtask才会被放到这里面来


for i in range(MAXHeight):
    subtasks = depths[i]
    for subtaskid in subtasks:
        
        number = re.findall(r'\d+', subtaskid)
        number = int(number[0]) if number else None
        subtask = steps_dict[number]
        
        # question 问题字符串
        # 交待解决任务
        sys_q = f"""You will be provided with a Programming Puzzle. Your task is to find an input that will make the program return True.
Here is the puzzle:\n{question}
I have broken this puzzle down into many easier subtasks. I will assign you sub-tasks one by one, and provide the results of the previous sub-tasks as a reference for your reasoning.
Please follow the logical sequence of our subtasks to solve this problem."""
        
        if len(answerDict)>0:
          answersSoFar = f"""\nSo far, the answers to the resolved sub-tasks are as follows: The format is SubtaskId: xxx; Subtask: xxx; Answer: xxx."""
          for key, value in answerDict.items():
            answersSoFar += f"""\nSubtaskId: {key}; Subtask: {answerDict[key]['subtask']}; Answer: {answerDict[key]['answer']}."""
            
          predecessors = search_Predecessors(int_edges, number)
          intersection = set(answerDict.keys()).intersection(set(predecessors))
          count = len(intersection)
          if count>0:
            answersSoFar += f"""\nAmong them, sub-tasks {predecessors} are directly related to this sub-task, so please pay special attention to them."""
        
          
        subask = f"""\nNow the subtask is: {subtask}
Based on the information above, please provide a concise and clear answer to this sub-task in one or two sentences.."""

        if len(answerDict)>0:
          query = answersSoFar+subtask
        else:
          query = subask

        Q = [
          {'role':'system', 'content':sys_q},
          {'role':'user', 'content':query},
        ]
              
        # print(subtaskid)
        # print(subtask)
        print('**********Question**********')
        print(Q)
        result = askChatGPT(Q, model=GPT_MODEL, temperature=1)
        print('Answer:', result)
        print('\n\n\n')
        
        answerDict[number] = {'subtask':subtask, 'answer':result}






**********Question**********
[{'role': 'system', 'content': 'You will be provided with a Programming Puzzle. Your task is to find an input that will make the program return True.\nHere is the puzzle:\ndef sat(x: float, a=7987622700):\n    return abs(x ** 2 - a) < 10 ** -3\nI have broken this puzzle down into many easier subtasks. I will assign you sub-tasks one by one, and provide the results of the previous sub-tasks as a reference for your reasoning.\nPlease follow the logical sequence of our subtasks to solve this problem.'}, {'role': 'user', 'content': "\nNow the subtask is:  Recall mathematical knowledge: 'x' is essentially the square root of 'a', within a small tolerance. \nBased on the information above, please provide a concise and clear answer to this sub-task in one or two sentences.."}]
Answer: Given that 'x' is essentially the square root of 'a' within a small tolerance, to find an appropriate 'x', one needs to calculate the square root of 'a' (7987622700) and ensure the re

In [119]:
 # 已经问完了所有的subtask,最后问一次得到最终的答案
Q.append({'role':'assistant', 'content':result})
Q.append({'role':'user', 'content':"""Now that all the sub-tasks have been completed, so what is the correct input?
Please give the input in the format of a string and just give the answer without any additional explanation or clarification."""})
finalResult = askChatGPT(Q, model=GPT_MODEL, temperature=1)
print(finalResult)

"89373.5533904092"


In [122]:
# 执行字符串形式的代码，使其在当前命名空间中定义函数
# 定义类型字典
from typing import Any, List
type_dict = {
    'str': str,
    'int': int,
    'float': float,
    'bool': bool,
    'List[int]': List[int],
    'List[str]': List[str],
    'List[float]': List[float],
    'List[bool]': List[bool],
    'List[List[int]]': List[List[int]],
    'List[List[float]]': List[List[float]],
    'List[List[str]]': List[List[str]],
    'List[List[List[int]]]': List[List[List[int]]]
}
# 根据ans_type_str将结果转换为相应的类型
def convert_to_type(type_str: str, value_str: str) -> Any:
    """
    根据给定的类型字符串和待转换的字符串,将其转换为相应的Python类型
    """
    if type_str in type_dict:
        python_type = type_dict[type_str]
        if type_str.startswith('List'):
            # 处理列表类型
            return eval(value_str)
        else:
            # 处理基本类型
            return python_type(value_str)
    else:
        raise ValueError(f"不支持的类型: {type_str}")

exec(question)

converted_result = convert_to_type(puzzles[i]['ans_type'], finalResult)

result = sat(converted_result)
print("函数运行结果为：", result)
print(result == True)

TypeError: unsupported operand type(s) for ** or pow(): 'str' and 'int'